In [ ]:
import numpy as np
np.random.seed(1)
from konlpy.tag import Komoran
import pandas as pd
from preprocessing import *

def extract_nouns(text):
    text = str(text)
    if len(text and text.strip()) > 1:
        text = replaceword(text)
        text = komoran.nouns(text)
        text = [x for x in text if x not in stopw]
        text = ' '.join(text)
    else:
        t = " "
    return text

def extract_pos(text):
    pos_used = ['NNG', 'NNP', 'NNB', 'VA']
    text = str(text)
    if len(text and text.strip()) > 1:
        text = replaceword(text)
        text = komoran.pos(text)
        pos_text = []
        for t in text:
            t = '/'.join(t)
            pos_text.append(t)
        
        text = [x for x in pos_text if x not in stopw]
        text = [x for x in text for p in pos_used if p in x]
    
        pos_text = []
        for t in text:
            t = t.replace('/VA', '다')
            t = t.replace('/NNG', '')
            pos_text.append(t)
        text = ' '.join(pos_text)
    
    else:
        t = " "
    return text

def morph_df(df):
    df = df.fillna(' ')
    pass_col = ['구분', '계열', '전공', '학년', '이름']
    
    for column in df:
        print(column)
        if column in pass_col : pass
        else:
            global stopw
            stop_tg_mask = (stopw_all['대상'] == '공통') | (stopw_all['대상'] == tg)
            stop_item_mask = (stopw_all['구분'] == '공통') | (stopw_all['구분'] == column)
            stopw = stopw_all.loc[stop_tg_mask, :]
            stopw = stopw.loc[stop_item_mask, :]['stopword'].unique().tolist()
            print(stopw)
            
            # 명사 추출
            # df[column] = df[column].apply(extract_nouns)
            
            # 다른 품사 추출
            df[column] = df[column].apply(extract_pos)
            
    return df

if __name__ == '__main__':
    
    # 사용자 사전 리스트
    dicpath = 'dict/userdict.txt'
    komoran = Komoran(userdic=dicpath)
    
    # 불용어 리스트
    stopw_all = pd.read_excel('dict/불용어, 용어통일, 사전 목록.xlsx', sheet_name = '불용어', usecols = ['대상', '구분', 'stopword'])
    
    # 데이터
    target = ['학부생', '대학원생']
    
    under_path = 'data/학부생 데이터 양식.xlsx'
    grad_path = 'data/대학원생 데이터 양식.xlsx'
    
    under_cat = ['기대 및 인식', '대학 생활 차원', '교육 및 학업 차원', '기타의견']
    grad_cat = ['기대 및 인식', '학술 및 연구활동', '교육 및 학업 차원', '기타의견']
    
    for tg in target:
        
        if tg == '학부생':
            filepath = under_path
            category = under_cat
            
        elif tg == '대학원생':
            filepath = grad_path
            category = grad_cat
            
        for cat in category:
            print(tg, '_', cat)
            df = pd.read_excel(filepath, sheet_name = cat)
            
            # 형태소 추출
            nouns_df = morph_df(df)
            nouns_df.to_excel('output/morph_{0}_{1}.xlsx'.format(tg, cat))